In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns


pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

import warnings
warnings.filterwarnings('ignore')

import gc
gc.collect()

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)


#model building
from sklearn.model_selection import train_test_split,KFold,cross_val_score,StratifiedKFold,GridSearchCV
from sklearn.metrics import mean_squared_log_error,accuracy_score,roc_auc_score, r2_score,mean_squared_error

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import lightgbm 

from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [2]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
train['gender']=np.where(train['gender']=='female',0,1)

In [3]:
train.head()

,session_id,startTime,endTime,ProductList,gender
0,u16159,15/12/14 18:11,15/12/14 18:12,A00002/B00003/C00006/D28435/;A00002/B00003/C00...,0
1,u10253,16/12/14 14:35,16/12/14 14:41,A00001/B00009/C00031/D29404/;A00001/B00009/C00...,1
2,u19037,01/12/14 15:58,01/12/14 15:58,A00002/B00001/C00020/D16944/,0
3,u14556,23/11/14 2:57,23/11/14 3:00,A00002/B00004/C00018/D10284/;A00002/B00004/C00...,0
4,u24295,17/12/14 16:44,17/12/14 16:46,A00001/B00001/C00012/D30805/;A00001/B00001/C00...,1


In [4]:
test.head()

,session_id,startTime,endTime,ProductList
0,u12112,08/12/14 13:36,08/12/14 13:36,A00002/B00003/C00006/D19956/
1,u19725,19/12/14 13:52,19/12/14 13:52,A00002/B00005/C00067/D02026/
2,u11795,01/12/14 10:44,01/12/14 10:44,A00002/B00002/C00004/D12538/
3,u22639,08/12/14 20:19,08/12/14 20:22,A00002/B00003/C00079/D22781/;A00002/B00003/C00...
4,u18034,15/12/14 19:33,15/12/14 19:33,A00002/B00001/C00010/D23419/


In [5]:
train['startTime']=pd.to_datetime(train['startTime'])
train['endTime']=pd.to_datetime(train['endTime'])
train['totalTime'] = (train['endTime']-train['startTime'])/np.timedelta64(1,'s')
train['dayName']=train['startTime'].dt.weekday_name
train['dayHour']=train['startTime'].dt.hour
train['totalTime']=train['totalTime'].abs()

test['startTime']=pd.to_datetime(test['startTime'])
test['endTime']=pd.to_datetime(test['endTime'])
test['totalTime'] = (test['endTime']-test['startTime'])/np.timedelta64(1,'s')
test['dayName']=test['startTime'].dt.weekday_name
test['dayHour']=test['startTime'].dt.hour
test['totalTime']=test['totalTime'].abs()

In [6]:
q1=train['totalTime'].quantile(0.25)
q2=train['totalTime'].quantile(0.75)
iqr=q2-q1
upper_bound=q2+1.5*(iqr)
lowerbound=q1-1.5*(iqr)

train['totalTime']=np.where(train['totalTime']>upper_bound,upper_bound,train['totalTime'])
test['totalTime']=np.where(test['totalTime']>upper_bound,upper_bound,test['totalTime'])

In [7]:
train.startTime.min(),train.startTime.max(),

(Timestamp('2014-01-12 00:07:00'), Timestamp('2014-12-22 23:54:00'))

In [8]:
train.head()

,session_id,startTime,endTime,ProductList,gender,totalTime,dayName,dayHour
0,u16159,2014-12-15 18:11:00,2014-12-15 18:12:00,A00002/B00003/C00006/D28435/;A00002/B00003/C00...,0,60.0,Monday,18
1,u10253,2014-12-16 14:35:00,2014-12-16 14:41:00,A00001/B00009/C00031/D29404/;A00001/B00009/C00...,1,300.0,Tuesday,14
2,u19037,2014-01-12 15:58:00,2014-01-12 15:58:00,A00002/B00001/C00020/D16944/,0,0.0,Sunday,15
3,u14556,2014-11-23 02:57:00,2014-11-23 03:00:00,A00002/B00004/C00018/D10284/;A00002/B00004/C00...,0,180.0,Sunday,2
4,u24295,2014-12-17 16:44:00,2014-12-17 16:46:00,A00001/B00001/C00012/D30805/;A00001/B00001/C00...,1,120.0,Wednesday,16


In [9]:
# train[train['dayName']=='Friday'].groupby(['gender'])['dayHour'].plot(kind='hist',bins=np.arange(0,24,1))
# plt.show()

In [10]:
# t = train.groupby(['dayName','gender']).size().unstack()
# t.columns=['female','male']
# t['female_per']=t['female']/t.sum(axis=1)
# t['male_per']=t['male']/t.sum(axis=1)
# t

# train.groupby(['gender'])['dayHour'].plot(kind='hist')
# plt.show()

In [11]:
#CHECKING CATEGORY WISE DISTRIBUTION
text=''
for t in train.ProductList:
    text+=t
category = list(set(re.findall(r'A[0-9]+',text)))

df = train.copy()
for name in category:
    df[name]=np.nan
    df[name]=np.nan

def categoryVerify(x,name):
    check=list(set(re.findall(r'A[0-9]+',x)))
    if name in check:
        return 1
    else:
        return 0

for name in category:
    df[name]=df['ProductList'].apply(lambda x: categoryVerify(x,name))

    
cols=category
cols.append('gender')
df=df[cols]
t=df.groupby(['gender']).agg('sum')
t.T

gender,0,1
A00006,97,22
A00009,20,11
A00005,192,41
A00001,392,1269
A00003,1508,152
A00010,22,33
A00008,13,11
A00002,6139,759
A00007,19,10
A00004,52,97


In [12]:
#CHECKING SUBCATEGORY WISE DISTRIBUTION
text=''
for t in train.ProductList:
    text+=t
subCategory = list(set(re.findall(r'B[0-9]+',text)))

df = train.copy()
for name in subCategory:
    df[name]=np.nan
    df[name]=np.nan

def subCategoryVerify(x,name):
    check=list(set(re.findall(r'B[0-9]+',x)))
    if name in check:
        return 1
    else:
        return 0

for name in subCategory:
    df[name]=df['ProductList'].apply(lambda x: subCategoryVerify(x,name))

    
cols=subCategory
cols.append('gender')
df=df[cols]
t=df.groupby(['gender']).agg('sum')
t.T

gender,0,1
B00001,1089,569
B00048,4,0
B00022,324,35
B00089,1,0
B00072,1,0
B00050,23,5
B00003,1408,183
B00055,6,5
B00042,1,4
B00074,0,1


In [13]:
#CHECKING subSubCategory WISE DISTRIBUTION
text=''
for t in train.ProductList:
    text+=t
subSubCategory = list(set(re.findall(r'C[0-9]+',text)))

df = train.copy()
for name in subSubCategory:
    df[name]=np.nan
    df[name]=np.nan

def subSubCategoryVerify(x,name):
    check=list(set(re.findall(r'C[0-9]+',x)))
    if name in check:
        return 1
    else:
        return 0

for name in subSubCategory:
    df[name]=df['ProductList'].apply(lambda x: subSubCategoryVerify(x,name))

    
cols=subSubCategory
cols.append('gender')
df=df[cols]
t=df.groupby(['gender']).agg('sum')
t.T

gender,0,1
C00137,5,4
C00260,1,0
C00216,1,1
C00148,20,1
C00046,186,24
C00192,6,0
C00134,15,1
C00377,2,0
C00252,3,0
C00115,12,1


In [14]:
text=''
for t in train.ProductList:
    text+=t
category = list(set(re.findall(r'A[0-9]+',text)))
subCategory = list(set(re.findall(r'B[0-9]+',text)))
subSubCategory = list(set(re.findall(r'C[0-9]+',text)))
product = list(set(re.findall(r'D[0-9]+',text)))


def catCount(x):
    n=len(re.findall(r'A[0-9]+',x))
    return n
def subCatCount(x):
    n=len(re.findall(r'B[0-9]+',x))
    return n
def subSubCatCount(x):
    n=len(re.findall(r'C[0-9]+',x))
    return n
def productCount(x):
    n=len(re.findall(r'D[0-9]+',x))
    return n

# train['numCategory']=train['ProductList'].apply(lambda x: catCount(x))
# train['numSubCategory']=train['ProductList'].apply(lambda x: subCatCount(x))
# train['numSubSubCategory']=train['ProductList'].apply(lambda x: subSubCatCount(x))
train['productCount']=train['ProductList'].apply(lambda x: productCount(x))

# test['numCategory']=test['ProductList'].apply(lambda x: catCount(x))
# test['numSubCategory']=test['ProductList'].apply(lambda x: subCatCount(x))
# test['numSubSubCategory']=test['ProductList'].apply(lambda x: subSubCatCount(x))
test['productCount']=test['ProductList'].apply(lambda x: productCount(x))

train['productTime'] = train['totalTime']/train['productCount']
test['productTime'] = test['totalTime']/test['productCount']

In [15]:
for name in category:
    train[name]=np.nan
    test[name]=np.nan

def catVerify(x,name):
    products=list(re.findall(r'A[0-9]+',x))
    if name in products:
        return products.count(name)
    else:
        return 0
    
for name in category:
    train[name]=train['ProductList'].apply(lambda x: catVerify(x,name))
    test[name]=test['ProductList'].apply(lambda x: catVerify(x,name))

#_____________________________________________________________________________
for name in subCategory:
    train[name]=np.nan
    test[name]=np.nan

def subCatVerify(x,name):
    products=list(re.findall(r'B[0-9]+',x))
    if name in products:
        return products.count(name)
    else:
        return 0
    
for name in subCategory:
    train[name]=train['ProductList'].apply(lambda x: subCatVerify(x,name))
    test[name]=test['ProductList'].apply(lambda x: subCatVerify(x,name))
    
    
#_____________________________________________________________________________
for name in subSubCategory:
    train[name]=np.nan
    test[name]=np.nan

def subSubCatVerify(x,name):
    products=list(re.findall(r'C[0-9]+',x))
    if name in products:
        return products.count(name)
    else:
        return 0
    
for name in subSubCategory:
    train[name]=train['ProductList'].apply(lambda x: subSubCatVerify(x,name))
    test[name]=test['ProductList'].apply(lambda x: subSubCatVerify(x,name))

In [16]:
train=pd.get_dummies(train,columns=['dayName'])
test=pd.get_dummies(test,columns=['dayName'])


In [17]:
X=train[[col for col in train.columns if col not in (['session_id','startTime','endTime','ProductList','gender',
                                                     ])]]
text = " ".join(X.columns)
# extra_col = list(set(re.findall(r'[C][0-9]+',text)))
# X = X[[col for col in X.columns if col not in extra_col]]
y=train.gender

test_=test.copy()
test_=test_[X.columns]

In [19]:
skf=StratifiedKFold(n_splits=10)
cv_score = []

lgb=lightgbm.LGBMClassifier(boosting_type='gbdt', 
                      objective='binary',
                      num_iteration=3000,
                      learning_rate=0.01,
                      max_depth=5,
                      colsample_bytree=0.9,
                      reg_alpha=0.0,
                      reg_lambda=0.0)
rfc=RandomForestClassifier(n_estimators=500)
lr=LogisticRegression(C=2)
xgb = XGBClassifier(n_estimators=1000,
                    max_depth=3,
                    learning_rate=0.1)

clfs=[xgb,lgb,rfc]


cols= ['xgb','lgb','rfc']
final=pd.DataFrame(columns=cols)
i=0
for clf_ in clfs:

    pred_test =np.zeros((len(test_),1))

    for train_index,test_index in skf.split(X,y):
        x_train,x_val = X.iloc[train_index],X.iloc[test_index]
        y_train,y_val = y.iloc[train_index],y.iloc[test_index]
        clf = clf_
        clf.fit(x_train,y_train)
        score = accuracy_score(y_val,clf.predict(x_val))
        cv_score.append(score)
        print(round(score,3),end=",")

        pred_test += clf.predict_proba(test_)[:,1].reshape(-1,1)
    pred_test=pred_test/10 
    final[cols[i]]=pred_test[:,0]
    i+=1
    print("\nmean_score: ",round(np.mean(cv_score),3))

0.882,0.875,0.87,0.856,0.882,0.867,0.879,0.864,0.888,0.867,
mean_score:  0.873
0.887,0.874,0.87,0.852,0.886,0.869,0.879,0.863,0.886,0.858,
mean_score:  0.873
0.861,0.856,0.841,0.84,0.854,0.841,0.849,0.849,0.871,0.852,
mean_score:  0.866


In [20]:
final['overall']=final.sum(axis=1)/3
final.head()

,xgb,lgb,rfc,overall
0,0.130294,0.128927,0.128694,0.129305
1,0.089075,0.092885,0.022718,0.068226
2,0.117222,0.122967,0.118803,0.119664
3,0.122508,0.156704,0.117200,0.132137
4,0.228066,0.224683,0.203141,0.218630


In [21]:
temp=test.copy()
temp['prob']=final.overall
temp['gender']=np.where(temp['prob']>=0.5,'male','female')
temp.index=temp.session_id
temp=temp[['gender']]
temp.to_csv('try6.csv')
temp.head()

,gender
session_id,
u12112,female
u19725,female
u11795,female
u22639,female
u18034,female
